# Intuição do algoritmo DEC (Deep Embedded Clustering)

### Instanciação da classe DEC

In [ ]:
from examples.empenhos_df import EMPENHOS
    
ds_train = EMPENHOS(
    train=True, testing_mode=True
)  # training dataset
ds_val = EMPENHOS(
    train=False, testing_mode=True
)  # evaluation dataset

In [5]:
import torch

model = torch.load("models/dec_model_full.pt", weights_only=False, map_location=torch.device('cpu'))
encoder = model.encoder

In [ ]:
# Criaremos o objeto DEC:
from ptdec.dec import DEC

model = DEC(cluster_number=51, hidden_dimension=56, encoder=encoder)

# Esse objeto possui 5 variáveis:
# self.encode
# self.hidden_dimension -> 56
# self.cluster_number   -> 51
# self.alpha            -> 1.0
# self.assignment = ClusterAssignment()

In [ ]:
model.assignment    # objeto da classe ClusterAssignment()
                    # Essa classe inicializa os clusters_centers a partir da função xavier_uniform
                    # e coloca esses centroides de shape [num_clusters, hidden_layer] dentro dos parâmetros de model

for param in model.parameters():
    print(param.shape)
    # Dentro dos parâmetros, nós temos as camadas do encoder (8, ao todo)
    # e uma camada [num_clusters, hidden_layer], que transforma o 
    # espaço dimensional de 'hidden_layer' para 'num_clusters'
    

torch.Size([1000, 387])
torch.Size([1000])
torch.Size([2000, 1000])
torch.Size([2000])
torch.Size([2000, 2000])
torch.Size([2000])
torch.Size([56, 2000])
torch.Size([56])
torch.Size([51, 56])


In [ ]:
# um batch de dados (num_samples, hidden_layer) será passado em forward() da classe ClusterAssignment()
# e no fim, teremos uma matriz que indicará as probabilidades de designação de clusters.
# return: [batch size, number of clusters]

# Intuição disponível em /intuition/clusterassignment.ipynb

### Treinamento do DEC

In [ ]:
from torch.optim import SGD

dec_optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
epochs_dec = 20
train(
    dataset=ds_train,
    model=model,
    epochs=epochs_dec,
    batch_size=10,
    optimizer=dec_optimizer,
    stopping_delta=0.000001,
    cuda=False,
    evaluate_batch_size=1024, # ?
)